<a href="https://colab.research.google.com/github/gpensabem/Learnings/blob/main/Svd_for_Dimensionality_Reduction_in_Signal_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
from scipy.io import wavfile
from google.colab import drive
drive.mount('/content/drive')

# Função para criar janelas sobrepostas de um sinal
def create_overlapping_windows(signal, window_size, overlap):
    step = window_size - overlap
    windows = []
    for i in range(0, len(signal) - window_size + 1, step):
        windows.append(signal[i:i+window_size])
    return np.array(windows)

# Carregar o sinal de áudio (exemplo de um arquivo WAV)
# Nota: Substitua 'path_to_file.wav' pelo caminho para seu arquivo de áudio
sample_rate, signal = wavfile.read('/content/drive/MyDrive/Aprendizados/Voicy_beep-de-censura.wav')

# Normalizar o sinal
signal = signal / np.max(np.abs(signal))

# Definir os parâmetros para a janela
window_size = 1024
overlap = 512

# Criar a matriz de janelas sobrepostas
A = create_overlapping_windows(signal, window_size, overlap)

# Aplicar SVD
U, S, VT = svd(signal)

# Selecionar o número de componentes principais
k = 50  # Você pode ajustar isso conforme necessário

# Truncar as matrizes
U_k = U[:, :k]
S_k = np.diag(S[:k])
VT_k = VT[:k, :]

# Reconstruir a matriz aproximada
A_k = np.dot(U_k, np.dot(S_k, VT_k))

# Converter a matriz aproximada de volta ao sinal
filtered_signal = np.zeros(len(signal))
count = np.zeros(len(signal))

step = window_size - overlap
for i in range(A_k.shape[0]):
    filtered_signal[i*step:i*step+window_size] += A_k[i]
    count[i*step:i*step+window_size] += 1

filtered_signal /= count

# Normalizar o sinal filtrado
filtered_signal = filtered_signal / np.max(np.abs(filtered_signal))

# Exibir o sinal original e o sinal filtrado
plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(signal)
plt.title('Sinal Original')
plt.subplot(2, 1, 2)
plt.plot(filtered_signal)
plt.title('Sinal Filtrado com SVD')
plt.show()

# Salvar o sinal filtrado em um arquivo WAV (opcional)
wavfile.write('filtered_signal.wav', sample_rate, (filtered_signal * 32767).astype(np.int16))
